In [3]:
import time
import pandas as pd

In [4]:
# Selenium imports
from selenium import webdriver
from selenium.common.exceptions import NoSuchElementException, ElementClickInterceptedException
from selenium.webdriver.common.by import By
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [22]:
def init_driver(path):    #path = '../driver/chromedriver.exe'
    # Initialize crome driver
    chrome_options = webdriver.ChromeOptions()
    chrome_options.add_argument('--disable-extensions')
    chrome_options.add_argument('--profile-directory=Default')
#     chrome_options.add_argument('--incognito')
    chrome_options.add_argument('--disable-pligins-discovery')
    chrome_options.add_argument('--start-maximized')
    
    browser = webdriver.Chrome(executable_path=path, chrome_options=chrome_options)
    
    return browser

In [23]:
def search_jobs(job_name:str, job_location:str, num_jobs:int, sleep_time, verbose):
    
    # initialize webdriver
    browser = init_driver(path='../driver/chromedriver.exe') # driver =  browser  path = '../driver/chromedriver.exe'
    url = 'https://www.glassdoor.com/Job/jobs.htm?suggestCount=0&suggestChosen=false&clickSource=searchBtn&typedKeyword=data+science&locT=N&locId=96&jobType=&context=Jobs&sc.keyword=data+science&dropdown=0'
    browser.get(url)
    
    job_title = WebDriverWait(browser, 10).until(EC.element_to_be_clickable((By.CSS_SELECTOR, "input[placeholder='Job Title, Keywords, or Company']")))
    location = WebDriverWait(browser, 10).until(EC.element_to_be_clickable((By.CSS_SELECTOR, "input[placeholder='Location']")))
    
    job_title.clear()
    location.clear()
    time.sleep(5)  # wait for 5 seconds
    
    job_title.send_keys(job_name)
    location.send_keys(job_location)
    time.sleep(3)
    
    press_submit = WebDriverWait(browser, 10).until(EC.element_to_be_clickable((By.CSS_SELECTOR, "button[type='submit']"))).click()
    jobs = []
    
    while len(jobs) < num_jobs:
###################CLOSE SIGN UP POP-UP#############################################        
        time.sleep(sleep_time)

        #Test for the "Sign Up" prompt and get rid of it.
        try:
            browser.find_element_by_class_name("selected").click()
        except ElementClickInterceptedException:
            pass

        time.sleep(.1)

        try:
            browser.find_element_by_css_selector('[alt="Close"]').click() #clicking to the X.
            print(' x out worked')
        except NoSuchElementException:
            print(' x out failed')
            pass
####################################################################################        
        # Going through each job in this page
        job_boxes = browser.find_elements_by_class_name("jl")
        for job_box in job_boxes:
            print("Progress: {}".format("" + str(len(jobs)) + "/" + str(num_jobs)))
            if len(jobs) >= num_jobs:
                break
                
            job_box.click()
            time.sleep(1)
            
            collected_successfully = False
            while not collected_successfully:
                try:
                    company_name = browser.find_element_by_xpath('.//div[@class="employerName"]').text
                    location = browser.find_element_by_xpath('.//div[@class="location"]').text
                    job_nu_naam = browser.find_element_by_xpath('.//div[contains(@class, "title")]').text
                    job_description = browser.find_element_by_xpath('.//div[@class="jobDescriptionContent desc"]').text  # The role para on webpage
                    collected_successfully = True
                except:
                    time.sleep(5)
                    
                try:
                    salary_estimate = browser.find_element_by_xpath('.//span[@class="css-1uyte9r css-hca4ks e1wijj242"]').text  # css-1uyte9r css-hca4ks e1wijj242
                except NoSuchElementException:
                    salary_estimate = -1
                    
                try:
                    rating = browser.find_element_by_xpath('.//span[@class="rating"]').text
                except NoSuchElementException:
                    rating = -1
                    
                if verbose:
                    print(f"Job Title: {job_nu_naam}")
                    print(f"Salary Estimate: {salart_estimate}")
                    print(f"Job Description: {job_description}")
                    print(f"Rating: {rating}")
                    print(f"Company Name: {company_name}")
                    print(f"Location: {location}")
                
                #Going to the Company tab...
                try:
                    browser.find_element_by_xpath('.//div[@data-tab-type="overview"]//span[text()="Company"]//following-sibling::*').click()
                    try:
                        headquarters = browser.find_element_by_xpath('.//div[@class="infoEntity"]//label[text()="Headquarters"]//following-sibling::*').text
                    except NoSuchElementException:
                        headquarters = -1
                        
                    try:
                        size = browser.find_element_by_xpath('.//div[@class="infoEntity"]//label[@text()="Size"]//following-sibling::*').text
                    except NoSuchElementException:
                        size = -1
                    
                    try:
                        founded = browser.find_element_by_xpath('.div[@class="infoEntity"]//label[@text()="Founded"]//following-siblings::*').text
                    except NoSuchElementException:
                        founded = -1
                    
                    try:
                        type_of_ownership = driver.find_element_by_xpath('.//div[@class="infoEntity"]//label[text()="Type"]//following-sibling::*').text
                    except NoSuchElementException:
                        type_of_ownership = -1
                
                    try:
                        industry = driver.find_element_by_xpath('.//div[@class="infoEntity"]//label[text()="Industry"]//following-sibling::*').text
                    except NoSuchElementException:
                        industry = -1
              
                    try:
                        sector = driver.find_element_by_xpath('.//div[@class="infoEntity"]//label[text()="Sector"]//following-sibling::*').text
                    except NoSuchElementException:
                        sector = -1

                    try:
                        revenue = driver.find_element_by_xpath('.//div[@class="infoEntity"]//label[text()="Revenue"]//following-sibling::*').text
                    except NoSuchElementException:
                        revenue = -1

                    try:
                        competitors = driver.find_element_by_xpath('.//div[@class="infoEntity"]//label[text()="Competitors"]//following-sibling::*').text
                    except NoSuchElementException:
                        competitors = -1

                except NoSuchElementException:  #Rarely, some job postings do not have the "Company" tab.
                    headquarters = -1
                    size = -1
                    founded = -1
                    type_of_ownership = -1
                    industry = -1
                    sector = -1
                    revenue = -1
                    competitors = -1
                
                if verbose:
                    print(f"Headquaters: {headquarters}")
                    print(f"Size: {size}")
                    print(f"Founded: {founded}")
                    print(f"Type of Ownership: {type_of_ownership}")
                    print(f"Industry: {industry}")
                    print(f"Sector:{sector}")
                    print(f"Revenue: {revenue}")
                    print(f"Competitors: {competitors}")
                    print("#$#$#$#$#$#$#$#$#$#$#$#$#$#$#$#$#$#$#$#$#$#$#$#$#$#$")
                    
                jobs.append({'Job Title':job_nu_naam,
                            "Salary Estimate" : salary_estimate,
                            "Job Description" : job_description,
                            "Rating" : rating,
                            "Company Name" : company_name,
                            "Location" : location,
                            "Headquarters" : headquarters,
                            "Size" : size,
                            "Founded" : founded,
                            "Type of ownership" : type_of_ownership,
                            "Industry" : industry,
                            "Sector" : sector,
                            "Revenue" : revenue,
                            "Competitors" : competitors})
            try:
                browser.find_element_by_xpath('.//li[@class="next"]//a//following-sibling::*').click()
            except NoSuchElementException:
                print(f"Scraping terminated before reaching target of jobs. Needed {num_jobs} , got {len(jobs)}")
        
        return pd.DataFrame(jobs)


In [26]:
df = search_jobs(job_name='data scientist', job_location='San Francisco, CA', num_jobs=15, sleep_time=15, verbose=False)
df

C:\Users\Saurabh\anaconda3\envs\webscrape\lib\site-packages\ipykernel_launcher.py:10: DeprecationWarning: use options instead of chrome_options
  # Remove the CWD from sys.path while we load stuff.


 x out worked
Progress: 0/15
Scraping terminated before reaching target of jobs. Needed 15 , got 1
Progress: 1/15
Scraping terminated before reaching target of jobs. Needed 15 , got 2
Progress: 2/15
Scraping terminated before reaching target of jobs. Needed 15 , got 3
Progress: 3/15
Scraping terminated before reaching target of jobs. Needed 15 , got 4
Progress: 4/15
Scraping terminated before reaching target of jobs. Needed 15 , got 5
Progress: 5/15
Scraping terminated before reaching target of jobs. Needed 15 , got 6
Progress: 6/15
Scraping terminated before reaching target of jobs. Needed 15 , got 7
Progress: 7/15
Scraping terminated before reaching target of jobs. Needed 15 , got 8
Progress: 8/15
Scraping terminated before reaching target of jobs. Needed 15 , got 9
Progress: 9/15
Scraping terminated before reaching target of jobs. Needed 15 , got 10
Progress: 10/15
Scraping terminated before reaching target of jobs. Needed 15 , got 11
Progress: 11/15
Scraping terminated before reach

,Job Title,Salary Estimate,Job Description,Rating,Company Name,Location,Headquarters,Size,Founded,Type of ownership,Industry,Sector,Revenue,Competitors
0,Data Scientist,$129K - $205K (Glassdoor est.),Checkr's mission is to build a fairer future b...,3.8,Checkr\n3.8,"San Francisco, CA",-1,-1,-1,-1,-1,-1,-1,-1
1,"Data Scientist, LiveRamp Safe Haven",$140K - $150K (Employer est.),"Data Scientist, LiveRamp Safe Haven\n\nLiveRam...",4.3,LiveRamp\n4.3,"San Francisco, CA",-1,-1,-1,-1,-1,-1,-1,-1
2,Senior Data Scientist,-1,About Fracta\nFracta.ai usesmachine learning a...,-1,Fracta,"Redwood City, CA",-1,-1,-1,-1,-1,-1,-1,-1
3,Director of Data Insights and Analytics,$99K - $193K (Glassdoor est.),Born from the belief that great things come to...,4.6,Yummly\n4.6,"San Carlos, CA",-1,-1,-1,-1,-1,-1,-1,-1
4,Senior Food Scientist,$94K - $152K (Glassdoor est.),Senior Food Scientist\nWe’re looking for a Pro...,3.5,Torani\n3.5,"South San Francisco, CA",-1,-1,-1,-1,-1,-1,-1,-1
5,Data Scientist,$100K - $161K (Glassdoor est.),Grammarly is continuing to grow our team durin...,5.0,Grammarly\n5.0,"San Francisco, CA",-1,-1,-1,-1,-1,-1,-1,-1
6,Applied Research Scientist,$87K - $176K (Glassdoor est.),Grammarly is continuing to grow our team durin...,5.0,Grammarly\n5.0,"San Francisco, CA",-1,-1,-1,-1,-1,-1,-1,-1
7,Data Scientist,-1,"Our Mission\nWe’re here to create a safer, hap...",4.7,Blue Owl\n4.7,"San Francisco, CA",-1,-1,-1,-1,-1,-1,-1,-1
8,Quantitative Research Associate,$52K - $109K (Glassdoor est.),Company Overview\nGlass Lewis is the world’s c...,4.0,Glass Lewis\n4.0,"San Francisco, CA",-1,-1,-1,-1,-1,-1,-1,-1
9,Data Scientist,$111K - $175K (Glassdoor est.),C3.ai is a leading enterprise AI software prov...,4.7,C3.ai\n4.7,"Redwood City, CA",-1,-1,-1,-1,-1,-1,-1,-1
